# 激光雷达与可见光相机配准

## 介绍
在激光雷达和可见光相机配准任务中，可见光相机的内参、畸变系数是已知的(通过张正友标定法)，因此问题的难点是激光雷达与可见光相机外参的标定。
除了本次实验采用的通过手动提取特征点的2D-3D的PnP算法外，还有一些自动化的外参标定算法。

### 一、文献1
在文献[1]中研究的是Velodyne机械式激光雷达和可见光相机的外参标定方法，由于机械式激光雷达具有水平扫描的特性，因此采用将标定板倾斜悬挂的方式，保证点云在每一条边上均有采样点。

<p align="center">
    <img src="../resources/paper/paper1_board.png" width=30%>
</p>

在获得每一条边的采样点后，通过直线拟合可以获得相邻边采样点构成的直线，进而得到标定板的角点(比手动在点云上提取要精确得多)。接着该论文探究了2D-3D配准的问题，指出在图像上手动提取特征点的方式不精确会导致重投影误差增大。


<p align="center">
    <img src="../resources/paper/paper1_board2.png" width=30%>
</p>

因此该文献采用在标定板上再贴上2维汉明码(ArUco markers)，由于标记可以对相机提供稳健的特征，并且知道其尺寸信息，因此类似棋盘格标定板计算出标记相对相机的外参，获得标记的3D坐标。因此把激光雷达-可见光相机配准问题转化成了一个3D-3D的配准问题，可以通过ICP算法的闭式解求解外参。 

## 二、文献2
### 2.1 Motivation
文献[2]将目前估计激光雷达和相机外参的方法分为不需要标记物的方法和需要标记物(文献[1-2])的方法(target-based approach)两种。

文献[2]说明当外参具有轻微的偏差的时候，可能导致5m外物体的重投影误差达到20cm，分析这是因为激光雷达点云的相对**稀疏性**及其**距离测量中的系统误差**。

<p align="center">
    <img src="../resources/paper/paper2_calibrate_result.png" width=30%>
</p>

距离测量的系统误差使得一个三维空间中的平面会具有“厚度”：

<p align="center">
    <img src="../resources/paper/paper2_system_error.png" width=30%>
</p>



### 2.2 提取角点算法
文献[3-4]提取角点(Vertices)的算法，会先提取标定物的重心和法向量，将标定板进行垂直投影后，再在二维平面上用RANSAC拟合直线，获得角点(文献[1]直接在三维空间中拟合直线)，并根据重心和法向量获得对应三维坐标。然而文献[2]指出[3-4]这种方法，会因为测距系统误差无法精确估计标定物的重心坐标和法线。

文献[2]工作的一大贡献就是设计了一个**显式地利用标定板几何信息，从而提取带有噪声(有厚度)标定平面点云角点**的方法。

<p align="center">
    <img src="../resources/paper/paper2_estimate_vertice.png" width=30%>
</p>

首先对目标点云$\mathscr{PC}$,变换到坐标系原点，并构造了如下优化问题

$$H_T^{L*} = \arg\min C(H_T^L(\mathscr{PC}))$$

其中$C(H_T^L(\mathscr{PC})) := \sum_{i=1}^N c(x_i, \epsilon) + c(y_i, d / 2)+ c(z_i, d / 2)$，其中d是标定物的长度，$\epsilon$衡量的是传感器的测距噪声水平(标准差)。函数$c(\cdot, \cdot)$定义如下所示

$$c(x, a):= \left\{
\begin{aligned}
 & min \{|x - a|, |x + a|\}  \qquad  &if |x| > a\\
& 0 \qquad &otherwise\\
\end{aligned}
\right.$$

在得到优化后的变换位姿后，点云中的物体角点为

$$X_i^{*} = H_L^{T*}(\bar{X_i}) \quad i\in\{1, 2, 3, 4\}$$

### 2.3 3D-2D匹配
文献[2]采用与图像角点进行3D-2D匹配的方式计算外参，然而它并没有采用PnP算法对重投影顶点非线性最小二乘的形式，而是采用最小化重投影多边形的IoU(类似目标检测中的将最小化MSE -> 最小化IoU)。

<p align="center">
    <img src="../resources/paper/paper2_IoU.png" width=30%>
</p>

计算IoU的方式如下：
1. 首先需要根据多边形的顶点计算面积，先把多边形顶点$\{\vec{X_i}\}$按照逆时针方向排列。则多边形面积为

$$A(V) = 1/2 * \sum_1^N det([\vec{X_i}, \vec{X_{i+1}}])$$

2. 计算IoU
$$IoU = \frac{A_I(V)}{A_L(V) + A_C(V) - A_I(V)}$$

最终优化问题为:

$$(R, t) = \arg\max_{R, t}IoU(A_L(V), A_C(V))$$

## 2.3 量化结果
量化结果显示，文献[2]提出的算法相对baseline[4]，降低了50%的重投影误差和70%的重投影方差。

<p align="center">
    <img src="../resources/paper/paper2_result.jpg" width=30%>
</p>

## 参考文献
[1] Dhall A , Chelani K , Radhakrishnan V , et al. LiDAR-Camera Calibration using 3D-3D Point correspondences[J]. 2017.

[2] J. -K. Huang and J. W. Grizzle, "Improvements to Target-Based 3D LiDAR to Camera Calibration," in IEEE Access, vol. 8, pp. 134101-134110, 2020, doi: 10.1109/ACCESS.2020.3010734.

[3] Q. Liao, Z. Chen, Y. Liu, Z. Wang, and M. Liu, “Extrinsic calibration of LiDAR and camera with polygon,” in 2018 IEEE International Conference on Robotics and Biomimetics (ROBIO). IEEE, 2018, pp. 200–205.

[4] L. Zhou, Z. Li, and M. Kaess, “Automatic extrinsic calibration of a camera and a 3D LiDAR using line and plane correspondences,” in 2018 IEEE/RSJ International Conference on Intelligent Robots and Systems(IROS). IEEE, 2018, pp. 5562–5569.